In [173]:
import codecs

# For status inquiry : WIN 205
STX=0x02
ADDR=0x80
WIN1=0x32
WIN2=0x30
WIN3=0x35
COM=0x30
ETX=0x03

# Pressure reading : WIN 812
STX=0x02
ADDR=0x80
WIN1=0x32
WIN2=0x30
WIN3=0x35
COM=0x30
ETX=0x03

CRCraw=hex(STX ^ ADDR ^ WIN1 ^ WIN2 ^ WIN3 ^ COM ^ ETX)
CRC1=codecs.encode(bytes(CRCraw[2].upper(),'utf-8'),"hex")
CRC2=codecs.encode(bytes(CRCraw[3].upper(),'utf-8'),"hex")

command=[STX, ADDR, WIN1, WIN2, WIN3, COM, ETX, int(CRC1,16), int(CRC2,16)]

a=''
for c in command:
#     print(str(hex(c)))
    a=a+str(hex(c))+' '
print(a)

0x2 0x80 0x32 0x30 0x35 0x30 0x3 0x38 0x36 


In [ ]:
0x2 0x80 0x38 0x39 0x30 0x30 0x3 0x38 0x30